## Lab#4, NLP@CGU Spring 2023

### This is due on 2023/04/20 16:00, commit to your github as a PDF (lab4.pdf) (File>Print>Save as PDF). 

#### IMPORTANT: After copying this notebook to your Google Drive, please paste a link to it below. To get a publicly-accessible link, hit the *Share* button at the top right, then click "Get shareable link" and copy over the result. If you fail to do this, you will receive no credit for this lab!

***LINK: *paste your link here****
####https://colab.research.google.com/drive/1bhXwXJXauEUEMyGO6qQmddcJwuHNL5CJ?usp=sharing
---

**Student ID**:B0928024

**Name**:莊靜修


#Word Embeddings for text classification
## 請訓練一個 kNN或是SVM 分類器來和 Google's Universal Sentence Encoder (a fixed-length 512-dimension embedding) 的分類結果比較

In [1]:
!wget -O Dcard.db https://github.com/cjwu/cjwu.github.io/raw/master/courses/nlp2023/lab4-Dcard-Dataset.db

--2023-04-24 05:17:02--  https://github.com/cjwu/cjwu.github.io/raw/master/courses/nlp2023/lab4-Dcard-Dataset.db
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/cjwu/cjwu.github.io/master/courses/nlp2023/lab4-Dcard-Dataset.db [following]
--2023-04-24 05:17:02--  https://raw.githubusercontent.com/cjwu/cjwu.github.io/master/courses/nlp2023/lab4-Dcard-Dataset.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 151552 (148K) [application/octet-stream]
Saving to: ‘Dcard.db’

Dcard.db            100%[===================>] 148.00K  --.-KB/s    in 0.02s   

2023-04-24 05:17:02 (6.50 MB/s) - ‘Dcard.db’ sa

In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("Dcard.db")  
df = pd.read_sql("SELECT * FROM Posts;", conn)
df

,createdAt,title,excerpt,categories,topics,forum_en,forum_zh
0,2022-03-04T07:54:19.886Z,專題需要數據🥺🥺幫填～,希望各位能花個20秒幫我填一下,,,dressup,穿搭
1,2022-03-04T07:42:59.512Z,#詢問 找衣服🥲,想找這套衣服🥲，但發現不知道該用什麼關鍵字找，（圖是草屯囝仔的校園演唱會截圖）,詢問,衣服 | 鞋子 | 衣物 | 男生穿搭 | 尋找,dressup,穿搭
2,2022-03-04T07:24:25.147Z,#黑特 網購50% FIFTY PERCENT請三思,因為文會有點長，先說結論是，50%是目前網購過的平台退貨最麻煩的一家，甚至我認為根本是刻意刁...,,黑特 | 網購 | 三思 | 退貨 | 售後服務,dressup,穿搭
3,2022-03-04T06:39:13.017Z,尋衣服,來源：覺得呱吉這襯衫好好看~~，或有人知道有類似的嗎,,衣服 | 尋找 | 日常穿搭 | 男生穿搭,dressup,穿搭
4,2022-03-04T06:28:06.137Z,#詢問 想問,各位，因為這個證件夾臺灣買不到，是美國outlet 的限量版貨，所以在以下的這間蝦皮上買，但...,詢問,穿搭 | 閒聊版 | 閒聊排解 | 假貨,dressup,穿搭
...,...,...,...,...,...,...,...
355,2022-03-03T03:41:10.972Z,開了新頻道,昨天上了第一支影片，之前有發過沒有線條的動畫影片，新的頻道改成有線條的，感覺大家好像比較喜歡...,,Youtuber | 頻道 | 有趣 | 日常 | 搞笑,youtuber,YouTuber
356,2022-03-03T02:26:58.821Z,估計某個YTUBER又有陰謀論可以寫了,今天全台灣大停電，應該過幾天就會有個戴面具的出來說，一定是中共……，我從上個影片就預測了……,,陰謀論 | Youtuber,youtuber,YouTuber
357,2022-03-02T21:25:51.080Z,#問 阿神和放火發生過什麼嗎？,想問有沒有人知道阿神和放火是認識還是有結過什麼仇之類的嗎？首先我個人基本沒關注過放火，但是最...,,Youtuber | 放火 | 阿神,youtuber,YouTuber
358,2022-03-02T20:33:47.713Z,#文長 我眼中的Rice&Shine,無意引戰，單純分享我的觀察與個人想法～這幾天看了Dcard幾篇關於Rice& Shine的貼...,,Riceandshine | Youtuber | 生活 | Vlog | youtuber板,youtuber,YouTuber


In [3]:
!pip3 install -q tensorflow_text
!pip3 install -q faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 14.7 MB/s eta 0:00:00


In [4]:
import tensorflow_hub as hub 
import numpy as np
import tensorflow_text
import faiss

embed_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [13]:
docid = 355
texts = "[" + df['title'] + '] [' + df['topics'] + '] ' + df['excerpt'] 
texts[docid]

'[開了新頻道] [Youtuber | 頻道 | 有趣 | 日常 | 搞笑] 昨天上了第一支影片，之前有發過沒有線條的動畫影片，新的頻道改成有線條的，感覺大家好像比較喜歡這種風格，試試看新的風格，影片內容主要是分享自己遇到的小故事，不知道這樣的頻道大家是否會想要看呢？喜歡的話也'

In [14]:
embeddings = embed_model(texts)
embed_arrays = np.array(embeddings)
index_arrays = df.index.values
topk = 10
# Step 1: Change data type
embeddings = embed_arrays.astype("float32")

# Step 2: Instantiate the index using a type of distance, which is L2 here
index = faiss.IndexFlatL2(embeddings.shape[1]) 

# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Step 4: Add vectors and their IDs
index.add_with_ids(embeddings, index_arrays)

D, I = index.search(np.array([embeddings[docid]]), topk)

plabel = df.iloc[docid]['forum_zh']

cols_to_show = ['title', 'excerpt', 'forum_zh']
plist = df.loc[I.flatten(), cols_to_show]

precision = 0
for index, row in plist.iterrows():
  if plabel == row["forum_zh"]:
    precision += 1

print("precision = ", precision/topk)
precision = 0

df.loc[I.flatten(), cols_to_show]

precision =  0.8


,title,excerpt,forum_zh
355,開了新頻道,昨天上了第一支影片，之前有發過沒有線條的動畫影片，新的頻道改成有線條的，感覺大家好像比較喜歡...,YouTuber
359,一個隨性系YouTube頻道,哈哈哈哈，沒錯我就是親友團來介紹一個我覺得很北七的頻道，現在觀看真的低的可憐，也沒事啦，就多...,YouTuber
330,《庫洛魔法使》（迷你）服裝製作,又來跟大家分享新的作品了~，頻道常常分享 {縫紉} {服裝製作} 等相關教學，大家對服裝製...,YouTuber
342,自己沒搞清楚狀況就不要亂黑勾惡,勾惡幫主在自己頻道簡介跟每部影片的下方都已經說明了，要分會會長以上才能看全部影片，這個說明已...,YouTuber
338,廚師系YouTuber,友人傳了這篇文給我，我一看，十大廚師系YouTuber，就猜一定有MASA，果不其然，榜上有...,YouTuber
243,毀我童年的家人,小時候都很喜歡看真珠美人魚和守護甜心，但是！！，每次晚餐看電視的時候，只要有播映到這種場景....,有趣
349,喜歡看寵物頻道的有嗎？🙋‍♂️,,YouTuber
332,#安利 翎週嗎 采翎,如題啦！ 最近突然超愛采翎，以前就很喜歡了，最近越來越愛~~，從之前的呱張新聞到新資料夾到翎...,YouTuber
340,超像Yoyo的啦～,先說，平常會看見習網美小吳的影片，但我不太會去追蹤YT的IG，然後在IG推薦的影片，就看到y...,YouTuber
263,大家熟悉的梗圖主角 昔與今,先貼幾張大家比較熟的，困惑的表情超傳神 🤣，用在比喻木頭男很適合🤣 跟貓咪一起用超好笑，生無...,有趣


#Implemement Your kNN or SVM classifier Here!
請比較分類結果中選出 topk 相近的筆數，並計算 forum_zh 是否都有在 query text 的 forum_zh 中

> [開了新頻道] [Youtuber | 頻道 | 有趣 | 日常 | 搞笑]


In [53]:
import collections
from collections import *
import jieba

tokenized = []

rec = collections.defaultdict(int)
for _, d in df.iterrows():
  short = []
  words = jieba.cut(d["title"] + d["excerpt"])
  for word in words:
      short.append(word)
  tokenized.append(short)
        
  for w in set(short):
      rec[w] += 1
print(len(tokenized))
print(tokenized)
print(rec)

360
[['專題', '需要', '數據', '🥺', '🥺', '幫填', '～', '希望', '各位', '能花個', '20', '秒', '幫', '我', '填', '一下'], ['#', '詢問', ' ', '找', '衣服', '🥲', '想', '找', '這套', '衣服', '🥲', '，', '但', '發現', '不', '知道', '該用', '什麼', '關鍵', '字', '找', '，', '（', '圖是', '草屯', '囝', '仔', '的', '校園', '演唱', '會', '截圖', '）'], ['#', '黑特', ' ', '網購', '50%', ' ', 'FIFTY', ' ', 'PERCENT', '請', '三思', '因為', '文會', '有', '點長', '，', '先說', '結論', '是', '，', '50%', '是', '目前', '網購過', '的', '平台', '退貨', '最麻煩', '的', '一家', '，', '甚至', '我', '認為', '根本', '是', '刻意', '刁難', '顧客', '，', '讓', '大家', '打', '消退', '貨的念頭', '。', '原本', '以為', '有', '實體', '店', '的', '平台', '網購', '會', '比', '較', '有', '保障', '，', '沒', '想到', '售後服務', '是', '遇過', '最糟', '的', '。', '—', '—', '以下', '正'], ['尋', '衣服', '來源', '：', '覺得', '呱吉', '這襯', '衫', '好好看', '~', '~', '，', '或', '有人', '知道', '有類', '似的', '嗎'], ['#', '詢問', ' ', '想問', '各位', '，', '因為', '這個', '證件夾', '臺', '灣買', '不到', '，', '是', '美國', 'outlet', ' ', '的', '限量版', '貨', '，', '所以', '在', '以下', '的', '這間', '蝦', '皮', '上', '買', '，', '但', '還是', '怕', '買', '到', '

In [54]:
from collections import Counter
import math

def calculate_tfidf(doc):
    count = Counter(doc)
    temp = {}
    for w, n in count.items():
        tf = n / len(doc)
        idf = len(tokenized) / rec[w]
        temp[w] = tf * math.log(idf, 10)
    
    return temp
    
    

tfidf = pd.DataFrame([calculate_tfidf(doc) for doc in tokenized])
    
tfidf = tfidf.fillna(0)
tfidf.head()

,專題,需要,數據,🥺,幫填,～,希望,各位,能花個,20,...,Ri,隨性系,哈哈哈哈,沒錯,親友,團來,北七,沒事,麥番,會不會
0,0.140955,0.116083,0.159769,0.206652,0.159769,0.05964,0.09232,0.072398,0.159769,0.103326,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.022276,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
label = df["forum_zh"]
label

0            穿搭
1            穿搭
2            穿搭
3            穿搭
4            穿搭
         ...   
355    YouTuber
356    YouTuber
357    YouTuber
358    YouTuber
359    YouTuber
Name: forum_zh, Length: 360, dtype: object

In [88]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(tfidf.values, label)
pred = knn.predict(tfidf.values)

In [89]:
def find(data):
  arr = knn.kneighbors(data,  n_neighbors=10, return_distance=False)
  precision = 0
  for i in arr[0]:
    if pred[i] == label.iloc[i]: precision += 1
  return precision

In [90]:
topk = 10

data = [tfidf.iloc[355].values]
precision = find(data)

# # DO NOT MODIFY THE BELOW LINE!
print("precision = ", precision/topk)


precision =  0.6
